In [33]:
%matplotlib inline
import matplotlib

import cv2
import grpc
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from tensorflow.python.framework import tensor_util
from modules.prnet import PRNet
from modules.prnet_image_cropper import PRNetImageCropper
from modules.face_detector import FaceDetector

import tensorflow as tf

# For plotting
from PRNet.utils.cv_plot import plot_kpt, plot_vertices

import time
import numpy as np


In [37]:
channel = grpc.insecure_channel('0.0.0.0:8500')
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

# Setup
PRNet.Setup()
FaceDetector.Setup()
PRNetImageCropper.Setup()


cap = cv2.VideoCapture("./tensorflow_face_detection/media/test.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))


out = None
frame_num = 1490
ind = 0


In [38]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

while frame_num:
    frame_num -= 1
    ret, image = cap.read()
    if ret == 0:
        break
    if out is None:
        out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(
            'M', 'J', 'P', 'G'), 24, (frame_width, frame_height))

    next_request = predict_pb2.PredictRequest()
    next_request.inputs['input_image'].CopyFrom(
      tf.make_tensor_proto(image))

    face_detector = FaceDetector()
    start_time = time.time()
    face_detector.PreProcess(next_request, stub)
    face_detector.Apply()
    prnet_request = face_detector.PostProcess()
    elapsed_time = time.time() - start_time
    print('face_detector time cost: {}'.format(elapsed_time))

    if "bounding_box" in prnet_request.inputs:
        start_time = time.time()
        prnet_image_cropper = PRNetImageCropper()
        prnet_image_cropper.PreProcess(prnet_request, stub)
        prnet_image_cropper.Apply()
        next_request = prnet_image_cropper.PostProcess()
        elapsed_time = time.time() - start_time
        print('prnet_image_cropper time cost: {}'.format(elapsed_time))

        prn = PRNet()
        start_time = time.time()
        prn.PreProcess(next_request, stub)
        prn.Apply()
        final_request = prn.PostProcess();
        elapsed_time = time.time() - start_time
        print('prnet time cost: {}'.format(elapsed_time))

        kpt = tensor_util.MakeNdarray(final_request.inputs["prnet_output"])
        vertices = tensor_util.MakeNdarray(final_request.inputs["vertices"])
#         out.write(plot_kpt(image, kpt))
        start_time = time.time()
        out.write(plot_vertices(np.zeros_like(image), vertices))
        elapsed_time = time.time() - start_time
        print('plot+write time cost: {}'.format(elapsed_time))
    else:
        out.write(image)


face_detector time cost: 0.0347039699554
prnet_image_cropper time cost: 0.0253999233246
prnet time cost: 0.0289440155029
plot+write time cost: 0.0545110702515
face_detector time cost: 0.0261850357056
prnet_image_cropper time cost: 0.0242760181427
prnet time cost: 0.0280609130859
plot+write time cost: 0.0586748123169
face_detector time cost: 0.0244319438934
prnet_image_cropper time cost: 0.0238440036774
prnet time cost: 0.0281050205231
plot+write time cost: 0.0560209751129
face_detector time cost: 0.0257108211517
prnet_image_cropper time cost: 0.023894071579
prnet time cost: 0.0299069881439
plot+write time cost: 0.0561900138855
face_detector time cost: 0.0244369506836
prnet_image_cropper time cost: 0.024092912674
prnet time cost: 0.0282330513
plot+write time cost: 0.059574842453
face_detector time cost: 0.025582075119
prnet_image_cropper time cost: 0.0242049694061
prnet time cost: 0.0281181335449
plot+write time cost: 0.0554311275482
face_detector time cost: 0.0263938903809
prnet_image_

plot+write time cost: 0.0554389953613
face_detector time cost: 0.0307869911194
prnet_image_cropper time cost: 0.0261931419373
prnet time cost: 0.0295028686523
plot+write time cost: 0.05504322052
face_detector time cost: 0.0318691730499
prnet_image_cropper time cost: 0.0258679389954
prnet time cost: 0.0303971767426
plot+write time cost: 0.0555140972137
face_detector time cost: 0.0267281532288
prnet_image_cropper time cost: 0.0241410732269
prnet time cost: 0.0279700756073
plot+write time cost: 0.0569529533386
face_detector time cost: 0.0278160572052
prnet_image_cropper time cost: 0.0246300697327
prnet time cost: 0.0285499095917
plot+write time cost: 0.0589411258698
face_detector time cost: 0.0252530574799
prnet_image_cropper time cost: 0.0252780914307
prnet time cost: 0.0284671783447
plot+write time cost: 0.0573830604553
face_detector time cost: 0.0254428386688
prnet_image_cropper time cost: 0.0241630077362
prnet time cost: 0.0278799533844
plot+write time cost: 0.0571529865265
face_detec

plot+write time cost: 0.0572509765625
face_detector time cost: 0.0257081985474
prnet_image_cropper time cost: 0.0237090587616
prnet time cost: 0.0277409553528
plot+write time cost: 0.055536031723
face_detector time cost: 0.0264508724213
prnet_image_cropper time cost: 0.0235450267792
prnet time cost: 0.0279421806335
plot+write time cost: 0.055862903595
face_detector time cost: 0.0269029140472
prnet_image_cropper time cost: 0.0245409011841
prnet time cost: 0.0279901027679
plot+write time cost: 0.055675983429
face_detector time cost: 0.0307021141052
prnet_image_cropper time cost: 0.0243980884552
prnet time cost: 0.0286240577698
plot+write time cost: 0.0555579662323
face_detector time cost: 0.027116060257
prnet_image_cropper time cost: 0.0254700183868
prnet time cost: 0.0293889045715
plot+write time cost: 0.0591139793396
face_detector time cost: 0.0240869522095
prnet_image_cropper time cost: 0.0236978530884
prnet time cost: 0.0279259681702
plot+write time cost: 0.0582318305969
face_detecto

In [39]:
cap.release()
out.release()
